In [1]:
#Depedencies
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
api_key = config.coin_api_key

In [2]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
    'listing_status':'active',
    'limit': 100,
    'sort': 'cmc_rank'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': api_key
}

session = Session()
session.headers.update(headers)
try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [3]:
#get the current datetime to add to the file
from datetime import datetime
now = datetime.now()
injest_date = now.strftime("%m/%d/%Y %H:%M:%S")

In [4]:
#grabs section of dictionary 
cmc_data = data['data']
#puts the dict into a DF, now need to expand nested platform column
cmc_df = pd.DataFrame.from_dict(cmc_data, orient='columns')
#expanded Datafram
full_df = pd.concat([cmc_df.drop(['platform'], axis= 1), cmc_df['platform'].apply(pd.Series)], axis= 1 )
#add column file name
full_df["File_Name"] = "Coin_Ranking_Dim"
#Add column for injested at 
full_df["injest_datetime"] = injest_date
full_df.head()

#turn dataframe to CSV
dfAsString = full_df.to_string(header=False, index=False)


 - write DF to parquet format
 `trial_write_to_parquet = test.to_parquet(headers = 'none')`
 - write DF to CSV format
`dfasCSV = test.to_csv()`
 - write DF to string format
`dfAsString = test.to_string(header=False, index=False)`


In [5]:
#this works, needed to send it as text !!
#it works too, had to save it as a CSV
from azure.storage.filedatalake import DataLakeServiceClient
# install the following package 
# pip install azure-storage-file-datalake 
# Get the below details from your storage account
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = config.container_name
directory_name = config.directory_name

service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", storage_account_name), credential=storage_account_key)

file_system_client = service_client.get_file_system_client(file_system=container_name)
dir_client = file_system_client.get_directory_client(directory_name)
dir_client.create_directory()
#set data to appropriate dataframe
data = dfAsString  


#make sure to change file type as well
file_client = dir_client.create_file("sampledata2.parquet")
file_client.append_data(data, 0, len(data))
file_client.flush_data(len(data))

{'date': datetime.datetime(2022, 3, 29, 22, 38, 50, tzinfo=datetime.timezone.utc),
 'etag': '"0x8DA11D4E57DCE83"',
 'last_modified': datetime.datetime(2022, 3, 29, 22, 38, 50, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': '01443c00-afb1-11ec-867f-74d83ee76d90',
 'request_id': 'ec3dd395-801f-00de-57bd-43bdab000000',
 'version': '2020-10-02'}